In [1]:
import requests
import json
import os
import foursquare
import pandas as pd
from tqdm import tqdm

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

#### Step 1: Set code to review full API

In [2]:
# Setup Foursquare API credentials 
api_key = os.environ.get('FOURSQUARE_API_KEY')

# Define the endpoint URL for the Foursqare request
endpoint = "https://api.foursquare.com/v3/places/search"

# Define the query parameters to include only the latitude and longitude fields
def foursquare_query(latitude, longitude):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000'
        }
    #Headers specific to FourSquare - used to specify wanted metadata - not typically found under all APIs. Found under https://location.foursquare.com/developer/reference/authentication
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
        }

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content



In [3]:
#### Step 2: Retrieve Foursquare results

In [4]:
#Retrieve the Foursquare query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = foursquare_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Foursquare query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, location_name):
    json_content = foursquare_query(latitude, longitude)
    poi_list = []
    for poi in json_content['results']:
        poi_dict = {}
        poi_dict['fsq_id'] = poi['fsq_id']
        poi_dict['location_name'] = location_name
        poi_dict['poi_type'] = [category['name'] for category in poi['categories']]
        poi_list.append(poi_dict)
    return poi_list

In [7]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

# extract the name, latitude, and longitude columns from the City Bike Data
names = bike_df['name']
latitudes = bike_df['latitude']
longitudes = bike_df['longitude']


# create an empty list to store the dictionaries
data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for fsq_id in fsq_ids:
        print(fsq_id['fsq_id'], fsq_id['location_name'], fsq_id['poi_type'])

# - LOSING LOCATION NAME AND VARIABLE WHEN RUNNING
# # read the file containing the locations
# bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

# # extract the name, latitude, and longitude columns from the City Bike Data
# names = bike_df['name']
# latitudes = bike_df['latitude']
# longitudes = bike_df['longitude']


# # create an empty list to store the dictionaries
# data_list = []

# # loop through each location and call the bike_radius_info function
# for i in tqdm(range(len(latitudes))):
#     fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
#     for fsq_id in fsq_ids:
#         data_list.append({'fsq_id': fsq_id['fsq_id'], 'location_name': fsq_id['location_name'], 'poi_type': fsq_id['poi_type']})



# # assign the list to a variable
# result = data_list



100%|██████████| 794/794 [09:47<00:00,  1.35it/s]


In [12]:
# loop through each location and call the bike_radius_info function
# for i in tqdm(range(len(latitudes))):
#     fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
print(fsq_ids)

[{'fsq_id': '4ac518dcf964a5204aa920e3', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['Chinese Restaurant']}, {'fsq_id': '4be2c8711dd22d7fb8c694bd', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['Night Club', 'Restaurant']}, {'fsq_id': '4bed2b6a91380f47617c9f18', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['Tobacco Store']}, {'fsq_id': '5980dbf30ff4f9479f731d60', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['French Restaurant']}, {'fsq_id': '5ce7fcd100b068002d2aa51f', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['Pastry Shop', 'Restaurant']}, {'fsq_id': '4e9b11e94690c21335507a89', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['Spa']}, {'fsq_id': '4e8d4f95e5e89c46f5bf0299', 'location_name': '003493 - South Audley Street, Mayfair', 'poi_type': ['Steakhouse']}, {'fsq_id': '4ac518e3f964a520c6aa20e3', 'location_name': '003493 - South Audle

### Put your parsed results into a DataFrame

In [9]:
POI_points_by_CityBikes = pd.DataFrame(result, columns=['fsq_id', 'Location', 'POI Type'])

In [10]:
POI_points_by_CityBikes

,fsq_id,Location,POI Type
0,4bd6e9305631c9b6fa8ca630,NaN,NaN
1,4ac518cdf964a520cca520e3,NaN,NaN
2,4cb5ee61f6616dcbfe42db08,NaN,NaN
3,5555d73f498eba1d92b0bd0a,NaN,NaN
4,5777b7f6498ec5cc16dce198,NaN,NaN
...,...,...,...
7935,4e9b11e94690c21335507a89,NaN,NaN
7936,4e8d4f95e5e89c46f5bf0299,NaN,NaN
7937,4ac518e3f964a520c6aa20e3,NaN,NaN
7938,50896c8be4b0f94297081043,NaN,NaN


# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

##### Set code to review full API

In [ ]:
https://api.yelp.com/v3

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating